In [263]:
# Function to replace characters in coures prerequisite based on pattern
def parse_prerequisite(rep, text):
    # Replacement
    rep = dict((re.escape(k), v) for k, v in rep.iteritems())
    pattern = re.compile("|".join(rep.keys()))
    text = pattern.sub(lambda m: rep[re.escape(m.group(0))], text)
    return text

In [356]:
# Function to split prerequisite and corequisite from coures prerequisites section
def split_prerequisite(text):
    # Has prerequisite(s)
    if len(text) > 1: 
        text_2 = text[1].split('.')
        
        # Switch course number to unicode
        uniString = text_2[0].encode('utf-8') 
        
        # Replace non-breaking space with space
        prerequisite_uniString = uniString.replace("\xc2\xa0", "") 
        
        # define desired first level replacements
        rep = {" or X": ";",
               ", ": "; ", 
               " and ": "; ",
               " / ": "; "} 
        
        parsed = parse_prerequisite(rep, prerequisite_uniString)
        
        # define desired second level replacements
        rep = {" or ": "; "}
        parsed = parse_prerequisite(rep, parsed)
        
        # define desired third level replacements
        rep = {";;": ";"}
        parsed = parse_prerequisite(rep, parsed)
        
        # Skip if string "units" in the prerequisite or corequisite
        if re.search('units', parsed) is None:   
            regex = re.compile('[A-Z]*\s?\d{1,}\w?')
        
            # Change prerequisite course list to ';' delimited string
            parsed_delimited = ';'.join(map(str, regex.findall(parsed.strip())))
        else:
            parsed_delimited = ""
        
        return parsed_delimited 

In [359]:
from bs4 import BeautifulSoup; #import beautiful soup scraper
import urllib2 #url library
import pandas as pd; #dataframe for final output
import re #regular expression package
from datetime import date

# Set url
url_1 = 'http://exploredegrees.stanford.edu/schoolofengineering/computerscience/#courseinventory'
url_2 = 'http://exploredegrees.stanford.edu/schoolofengineering/electricalengineering/#courseinventory'

page_1 = urllib2.urlopen(url_1) #open the url
soup_1 = BeautifulSoup(page_1.read(), "lxml") #read in beautiful soup

page_2 = urllib2.urlopen(url_2) #open the url
soup_2 = BeautifulSoup(page_2.read(), "lxml") #read in beautiful soup

course_block_title_1 = [s.get_text() for s in soup_1.find_all("p", {"class":"courseblocktitle"}, "strong")] # Get course black title
course_number_title_1 = course_block_title_1

course_block_title_2 = [s.get_text() for s in soup_2.find_all("p", {"class":"courseblocktitle"}, "strong")] # Get course black title
course_number_title_2 = course_block_title_2

course_block_desc_1 = [s.get_text() for s in soup_1.find_all("p", {"class":"courseblockdesc"})] # Get course requsites
course_block_desc_2 = [s.get_text() for s in soup_2.find_all("p", {"class":"courseblockdesc"})] # Get course requsites

# Create empty lists for each course attribute
courseNumber = [] 
courseTitle = []
prerequisite = []
dateScrape = []
regex = []
    
# Append course number and couse title from courses in Computer Science Department
for each in course_number_title_1:
    course = each.split('.') # Split at period
    
    # Switch course number to unicode
    uniString = course[0].encode('utf-8') 
    
    # Replace non-breaking space with space
    course_uniString = uniString.replace("\xc2\xa0", "") 
    
    # Append to the course number list
    courseNumber.append(course_uniString.strip()) 
    
    # Switch course title to unicode and append to course title list
    courseTitle.append(course[1].encode('utf-8').strip()) 

# Append course number and couse title from courses in Electrical Engineering Department
for each in course_number_title_2:
    course = each.split('.') # Split at period
    
    # Switch course number to unicode
    uniString = course[0].encode('utf-8') 
    
    # Replace non-breaking space with space
    course_uniString = uniString.replace("\xc2\xa0", "") 
    
    # Append to the course number list
    courseNumber.append(course_uniString.strip()) 
    
    # Switch course title to unicode and append to course title list
    courseTitle.append(course[1].encode('utf-8').strip()) 

# Append course description from courses in Computer Science Department
for each in course_block_desc_1:
    # Parse prerequisite using delimiter ':'
    split_1 = re.split(r'Prerequisite[s]?[:]\s*', each)
    
    # Parse corequisite using delimiter ':'
    coreq_split_1 = re.split(r'Corequisite[s]?[:]\s*', each)
    
    # Append the prerequisite and corequisite course list to the course
    if (split_prerequisite(split_1) is not None) and \
        (split_prerequisite(coreq_split_1) is not None):
        prerequisite.append(split_prerequisite(split_1) + \
                            split_prerequisite(coreq_split_1)) 
        
    elif (split_prerequisite(split_1) is not None) and \
          (split_prerequisite(coreq_split_1) is None):
        prerequisite.append(split_prerequisite(split_1)) 
          
    elif (split_prerequisite(split_1) is None) and \
          (split_prerequisite(coreq_split_1) is not None):
        prerequisite.append(split_prerequisite(coreq_split_1))
          
    else:
        prerequisite.append("") # Store empty string if no prerequisite or corequisite

# Append course number and couse title from courses in Electrical Engineering Department
for each in course_block_desc_2:
    # Parse prerequisite using delimiter ':'
    split_1 = re.split(r'Prerequisite[s]?[:]\s*', each)
    
    # Parse corequisite using delimiter ':'
    coreq_split_1 = re.split(r'Corequisite[s]?[:]\s*', each)
    
    # Append the prerequisite and corequisite course list to the course
    if (split_prerequisite(split_1) is not None) and \
        (split_prerequisite(coreq_split_1) is not None):
        prerequisite.append(split_prerequisite(split_1) + \
                            split_prerequisite(coreq_split_1)) 
        
    elif (split_prerequisite(split_1) is not None) and \
          (split_prerequisite(coreq_split_1) is None):
        prerequisite.append(split_prerequisite(split_1)) 
          
    elif (split_prerequisite(split_1) is None) and \
          (split_prerequisite(coreq_split_1) is not None):
        prerequisite.append(split_prerequisite(coreq_split_1))
          
    else:
        prerequisite.append("") # Store empty string if no prerequisite or corequisite
      
# Create a list of date for the date of scrape
dateScrape = [date.today().strftime('%m/%d/%Y')] * len(courseNumber)

# Set department
department_1 = ['Computer Science'] * len(course_number_title_1)
department_2 = ['Electrical Engineering'] * len(course_number_title_2)

# Set university
university = "Stanford"

print "Add all data to a dataframe"
output = pd.DataFrame(dateScrape) #create a dataframe for outputs
output.columns = ['Date of Scraping']
output['University'] = university
output['Department'] = department_1 + department_2
output['Course Number'] = courseNumber
output['Course Title'] = courseTitle
output['Prerequisite'] = prerequisite

# Output data to a csv file
print "Output data to file"
output.to_csv('wcao-INFX575-PS1_dataset.csv', index=False) # Skip index column

Add all data to a dataframe
Output data to file
